## Refactor Scraping Logic

Encapsulate the web scraping part into a dedicated function `scrape_all_quotes` that returns the list of quotes.


In [1]:
import requests
from bs4 import BeautifulSoup
from csv import writer
from time import sleep
from random import choice

def scrape_all_quotes():
    # list to store scraped data
    all_quotes = []

    # this part of the url is constant
    base_url = "http://quotes.toscrape.com/"

    # this part of the url will keep changing
    url = "/page/1"

    while url:
        # concatenating both urls
        # making request
        res = requests.get(f"{base_url}{url}")
        print(f"Now Scraping {base_url}{url}")
        soup = BeautifulSoup(res.text, "html.parser")

        # extracting all elements
        quotes = soup.find_all(class_="quote")

        for quote in quotes:
            all_quotes.append({
                "text": quote.find(class_="text").get_text(),
                "author": quote.find(class_="author").get_text(),
                "bio-link": quote.find("a")["href"]
            })
        next_btn = soup.find(class_="next")
        url = next_btn.find("a")["href"] if next_btn else None
        sleep(2)
    return all_quotes

# Call the function to scrape all quotes
all_quotes_data = scrape_all_quotes()

# The game-related code is removed as per the instructions.


Now Scraping http://quotes.toscrape.com//page/1
Now Scraping http://quotes.toscrape.com//page/2/
Now Scraping http://quotes.toscrape.com//page/3/
Now Scraping http://quotes.toscrape.com//page/4/
Now Scraping http://quotes.toscrape.com//page/5/
Now Scraping http://quotes.toscrape.com//page/6/
Now Scraping http://quotes.toscrape.com//page/7/
Now Scraping http://quotes.toscrape.com//page/8/
Now Scraping http://quotes.toscrape.com//page/9/
Now Scraping http://quotes.toscrape.com//page/10/


## Improve Scraping Robustness and Fix Warning

Add error handling for `requests.get` to gracefully handle network issues. The warning related to `_class` has already been fixed in the previous step.


In [2]:
import requests
from bs4 import BeautifulSoup
from csv import writer
from time import sleep
from random import choice

def scrape_all_quotes():
    # list to store scraped data
    all_quotes = []

    # this part of the url is constant
    base_url = "http://quotes.toscrape.com/"

    # this part of the url will keep changing
    url = "/page/1"

    while url:
        try:
            # concatenating both urls
            # making request with timeout
            full_url = f"{base_url}{url}"
            res = requests.get(full_url, timeout=10)
            res.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
            print(f"Now Scraping {full_url}")
            soup = BeautifulSoup(res.text, "html.parser")

            # extracting all elements
            quotes = soup.find_all(class_="quote")

            for quote in quotes:
                all_quotes.append({
                    "text": quote.find(class_="text").get_text(),
                    "author": quote.find(class_="author").get_text(),
                    "bio-link": quote.find("a")["href"]
                })
            next_btn = soup.find(class_="next")
            url = next_btn.find("a")["href"] if next_btn else None
            sleep(2)
        except requests.exceptions.RequestException as e:
            print(f"Error scraping {full_url}: {e}")
            url = None # Terminate the loop on error
    return all_quotes

# Call the function to scrape all quotes
all_quotes_data = scrape_all_quotes()

# The game-related code is removed as per the instructions.

Now Scraping http://quotes.toscrape.com//page/1
Now Scraping http://quotes.toscrape.com//page/2/
Now Scraping http://quotes.toscrape.com//page/3/
Now Scraping http://quotes.toscrape.com//page/4/
Now Scraping http://quotes.toscrape.com//page/5/
Now Scraping http://quotes.toscrape.com//page/6/
Now Scraping http://quotes.toscrape.com//page/7/
Now Scraping http://quotes.toscrape.com//page/8/
Now Scraping http://quotes.toscrape.com//page/9/
Now Scraping http://quotes.toscrape.com//page/10/


## Refactor Game Logic

Encapsulate the quote guessing game logic into a function `play_quote_game` that takes the scraped list of quotes as input.


In [3]:
import requests
from bs4 import BeautifulSoup
from csv import writer
from time import sleep
from random import choice

# This part of the URL is constant and used by both scraping and game logic
base_url = "http://quotes.toscrape.com/"

def scrape_all_quotes():
    # list to store scraped data
    all_quotes = []

    # this part of the url will keep changing
    url = "/page/1"

    while url:
        try:
            # concatenating both urls
            # making request with timeout
            full_url = f"{base_url}{url}"
            res = requests.get(full_url, timeout=10)
            res.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
            print(f"Now Scraping {full_url}")
            soup = BeautifulSoup(res.text, "html.parser")

            # extracting all elements
            quotes = soup.find_all(class_="quote")

            for quote in quotes:
                all_quotes.append({
                    "text": quote.find(class_="text").get_text(),
                    "author": quote.find(class_="author").get_text(),
                    "bio-link": quote.find("a")["href"]
                })
            next_btn = soup.find(class_="next")
            url = next_btn.find("a")["href"] if next_btn else None
            sleep(2)
        except requests.exceptions.RequestException as e:
            print(f"Error scraping {full_url}: {e}")
            url = None # Terminate the loop on error
    return all_quotes

def play_quote_game(quotes_data):
    quote = choice(quotes_data)
    remaining_guesses = 4
    print("Here's a quote:  ")
    print(quote["text"])

    guess = ''
    while guess.lower() != quote["author"].lower() and remaining_guesses > 0:
        guess = input(
            f"Who said this quote? Guesses remaining {remaining_guesses}")

        if guess.lower() == quote["author"].lower():
            print("CONGRATULATIONS!!! YOU GOT IT RIGHT")
            break
        remaining_guesses -= 1

        if remaining_guesses == 3:
            res = requests.get(f"{base_url}{quote['bio-link']}")
            soup = BeautifulSoup(res.text, "html.parser")
            birth_date = soup.find(class_="author-born-date").get_text()
            birth_place = soup.find(class_="author-born-location").get_text()
            print(
                f"Here's a hint: The author was born on {birth_date}{birth_place}")

        elif remaining_guesses == 2:
            print(
                f"Here's a hint: The author's first name starts with: {quote['author'][0]}")

        elif remaining_guesses == 1:
            # Split by space and take the first character of the second word (last name)
            author_parts = quote["author"].split(" ")
            if len(author_parts) > 1: # Ensure there is a last name
                last_initial = author_parts[1][0]
                print(
                    f"Here's a hint: The author's last name starts with: {last_initial}")
            else:
                print("Here's a hint: The author only has one name, and its first letter is the hint already given.")

        else:
            print(
                f"Sorry, you ran out of guesses. The answer was {quote['author']}")

# Call the function to scrape all quotes
all_quotes_data = scrape_all_quotes()

# Call the function to play the game
if all_quotes_data:
    play_quote_game(all_quotes_data)
else:
    print("No quotes were scraped. Cannot start the game.")

Now Scraping http://quotes.toscrape.com//page/1
Now Scraping http://quotes.toscrape.com//page/2/
Now Scraping http://quotes.toscrape.com//page/3/
Now Scraping http://quotes.toscrape.com//page/4/
Now Scraping http://quotes.toscrape.com//page/5/
Now Scraping http://quotes.toscrape.com//page/6/
Now Scraping http://quotes.toscrape.com//page/7/
Now Scraping http://quotes.toscrape.com//page/8/
Now Scraping http://quotes.toscrape.com//page/9/
Now Scraping http://quotes.toscrape.com//page/10/
Here's a quote:  
“I am good, but not an angel. I do sin, but I am not the devil. I am just a small girl in a big world trying to find someone to love.”
Who said this quote? Guesses remaining 4anna morgan
Here's a hint: The author was born on June 01, 1926in The United States
Who said this quote? Guesses remaining 3marilyn monroe
CONGRATULATIONS!!! YOU GOT IT RIGHT


## Implement Multiple Game Rounds and Scoring

Modify the `play_quote_game` function to allow the user to play multiple rounds, keep track of their score, and display the final score at the end of the game.


In [4]:
import requests
from bs4 import BeautifulSoup
from csv import writer
from time import sleep
from random import choice

# This part of the URL is constant and used by both scraping and game logic
base_url = "http://quotes.toscrape.com/"

def scrape_all_quotes():
    # list to store scraped data
    all_quotes = []

    # this part of the url will keep changing
    url = "/page/1"

    while url:
        try:
            # concatenating both urls
            # making request with timeout
            full_url = f"{base_url}{url}"
            res = requests.get(full_url, timeout=10)
            res.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
            print(f"Now Scraping {full_url}")
            soup = BeautifulSoup(res.text, "html.parser")

            # extracting all elements
            quotes = soup.find_all(class_="quote")

            for quote in quotes:
                all_quotes.append({
                    "text": quote.find(class_="text").get_text(),
                    "author": quote.find(class_="author").get_text(),
                    "bio-link": quote.find("a")["href"]
                })
            next_btn = soup.find(class_="next")
            url = next_btn.find("a")["href"] if next_btn else None
            sleep(2)
        except requests.exceptions.RequestException as e:
            print(f"Error scraping {full_url}: {e}")
            url = None # Terminate the loop on error
    return all_quotes

def play_quote_game(quotes_data):
    score = 0
    play_again = 'y'

    while play_again.lower() == 'y':
        quote = choice(quotes_data)
        remaining_guesses = 4
        print("\nHere's a quote:  ")
        print(quote["text"])

        guess = ''
        while guess.lower() != quote["author"].lower() and remaining_guesses > 0:
            guess = input(
                f"Who said this quote? Guesses remaining {remaining_guesses}")

            if guess.lower() == quote["author"].lower():
                print("CONGRATULATIONS!!! YOU GOT IT RIGHT")
                score += 1
                break
            remaining_guesses -= 1

            if remaining_guesses == 3:
                # Fetch author's birth details for the hint
                try:
                    author_bio_url = f"{base_url}{quote['bio-link']}"
                    res = requests.get(author_bio_url, timeout=5)
                    res.raise_for_status()
                    soup = BeautifulSoup(res.text, "html.parser")
                    birth_date = soup.find(class_="author-born-date").get_text()
                    birth_place = soup.find(class_="author-born-location").get_text()
                    print(
                        f"Here's a hint: The author was born on {birth_date}{birth_place}")
                except requests.exceptions.RequestException as e:
                    print(f"Could not fetch author bio for hint: {e}")
                    print("Here's a hint: I couldn't get more info about the author's birth.")
            elif remaining_guesses == 2:
                print(
                    f"Here's a hint: The author's first name starts with: {quote['author'][0]}")

            elif remaining_guesses == 1:
                author_parts = quote["author"].split(" ")
                if len(author_parts) > 1: # Ensure there is a last name
                    last_initial = author_parts[1][0]
                    print(
                        f"Here's a hint: The author's last name starts with: {last_initial}")
                else:
                    print("Here's a hint: The author only has one name, and its first letter is the hint already given.")

            else:
                print(
                    f"Sorry, you ran out of guesses. The answer was {quote['author']}")

        print(f"Current score: {score}")
        play_again = input("Do you want to play another round? (y/n) ")

    print(f"\nGame Over! Your final score is: {score}")

# Call the function to scrape all quotes
all_quotes_data = scrape_all_quotes()

# Call the function to play the game
if all_quotes_data:
    play_quote_game(all_quotes_data)
else:
    print("No quotes were scraped. Cannot start the game.")

Now Scraping http://quotes.toscrape.com//page/1
Now Scraping http://quotes.toscrape.com//page/2/
Now Scraping http://quotes.toscrape.com//page/3/
Now Scraping http://quotes.toscrape.com//page/4/
Now Scraping http://quotes.toscrape.com//page/5/
Now Scraping http://quotes.toscrape.com//page/6/
Now Scraping http://quotes.toscrape.com//page/7/
Now Scraping http://quotes.toscrape.com//page/8/
Now Scraping http://quotes.toscrape.com//page/9/
Now Scraping http://quotes.toscrape.com//page/10/

Here's a quote:  
“Love does not begin and end the way we seem to think it does. Love is a battle, love is a war; love is a growing up.”
Who said this quote? Guesses remaining 4keanu reeves
Here's a hint: The author was born on August 02, 1924in Harlem, New York, The United States
Who said this quote? Guesses remaining 3will smith
Here's a hint: The author's first name starts with: J
Who said this quote? Guesses remaining 2jackson
Here's a hint: The author's last name starts with: B
Who said this quote? 